In [1]:
from keras.applications import VGG16
#Load the VGG model
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

Using TensorFlow backend.
C:\Users\Admin\AppData\Local\Continuum\anacond\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Admin\AppData\Local\Continuum\anacond\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Admin\AppData\Local\Continuum\anacond\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Admin

In [2]:
from keras.preprocessing.image import ImageDataGenerator

In [3]:
for layer in vgg_conv.layers[:-8]:
    layer.trainable = False
 
# Check the trainable status of the individual layers
for layer in vgg_conv.layers:
    print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x0000015E9D89EE88> False
<keras.layers.convolutional.Conv2D object at 0x0000015EA6EB70C8> False
<keras.layers.convolutional.Conv2D object at 0x0000015EA6EB7888> False
<keras.layers.pooling.MaxPooling2D object at 0x0000015EA6EFFE48> False
<keras.layers.convolutional.Conv2D object at 0x0000015EA6F08388> False
<keras.layers.convolutional.Conv2D object at 0x0000015EA6F1BA48> False
<keras.layers.pooling.MaxPooling2D object at 0x0000015EA6F73A08> False
<keras.layers.convolutional.Conv2D object at 0x0000015EA6F94448> False
<keras.layers.convolutional.Conv2D object at 0x0000015EA6F87E08> False
<keras.layers.convolutional.Conv2D object at 0x0000015EA6FD6248> False
<keras.layers.pooling.MaxPooling2D object at 0x0000015EA7007C88> False
<keras.layers.convolutional.Conv2D object at 0x0000015EA6FEE4C8> True
<keras.layers.convolutional.Conv2D object at 0x0000015EA701C3C8> True
<keras.layers.convolutional.Conv2D object at 0x0000015EA7029AC8> True
<keras.

In [8]:
from keras import models
from keras import layers
from keras import optimizers
 
# Create the model
model = models.Sequential()
 
# Add the vgg convolutional base model
model.add(vgg_conv)
 
# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))
 
# Show a summary of the model. Check the number of trainable parameters
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_2 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 1024)              25691136  
_________________________________________________________________
dense_5 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 10)                5130      
Total params: 40,935,754
Trainable params: 39,200,266
Non-trainable params: 1,735,488
__________________________________

In [9]:
train_dir = 'PlantVillage/'
validation_dir = 'PlantVillage/'

In [10]:
import numpy as np
from random import shuffle

In [11]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
#       rotation_range=20,
#       width_shift_range=0.2,
#       height_shift_range=0.2,
#       horizontal_flip=True,
#       fill_mode='nearest'
)
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 16
val_batchsize = 10
 
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(224, 224),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=True)

Found 16011 images belonging to 10 classes.
Found 16011 images belonging to 10 classes.


In [12]:
print(train_generator.class_indices)

{'Tomato_Bacterial_spot': 0, 'Tomato_Early_blight': 1, 'Tomato_Late_blight': 2, 'Tomato_Leaf_Mold': 3, 'Tomato_Septoria_leaf_spot': 4, 'Tomato_Spider_mites_Two_spotted_spider_mite': 5, 'Tomato__Target_Spot': 6, 'Tomato__Tomato_YellowLeaf__Curl_Virus': 7, 'Tomato__Tomato_mosaic_virus': 8, 'Tomato_healthy': 9}


In [13]:
# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])
# Train the model
history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=3,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1)


Epoch 1/3
1001/1000 [==============================] - 27413s 27s/step - loss: 0.8794 - acc: 0.7114 - val_loss: 0.1173 - val_acc: 0.9000
Epoch 2/3
1001/1000 [==============================] - 28840s 29s/step - loss: 0.4217 - acc: 0.8808 - val_loss: 0.0000e+00 - val_acc: 0.9207
Epoch 3/3
1001/1000 [==============================] - 31344s 31s/step - loss: 0.3836 - acc: 0.9056 - val_loss: 1.1228 - val_acc: 0.9260


In [14]:
import cv2

In [15]:
img3 = cv2.imread('./Test Images sample/Tom_hl.JPG')
img3 = cv2.resize(img3,(224,224))
img3 = np.reshape(img3,[1,224,224,3])

In [16]:
disease = model.predict_classes(img3)
print(disease)

[9]


In [17]:
model.save('plant_disease_final.h5')